<a href="https://colab.research.google.com/github/NoorFatimaMemon/DeepLearning/blob/main/26_Intro_to_Autoencoders%20/26_Intro_to_Autoencoders_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Autoencoders with Pytorch**

In [1]:
# Importing libraries
import os
import torch
from torch import nn
from torch.autograd import Variable      # wraps tensors for autograd
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [4]:
# Image Conversion Function
def to_img(x):
    x = 0.5 * (x + 1)           # Rescale from [-1, 1] to [0, 1]
    x = x.clamp(0, 1)           # Clamp values to ensure they’re within [0,1]
    x = x.view(x.size(0), 1, 28, 28)  # Reshape to image format (batch, channel, height, width)
    return x

In [5]:
# Hyperparameters and Dataset Setup
num_epochs = 100
batch_size = 128
learning_rate = 1e-3
# Transforms images to tensors and normalizes pixel values from [0, 1] to [-1, 1].
img_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

In [6]:
dataset = MNIST('./data', transform=img_transform, download = True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 12.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 340kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.17MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 11.9MB/s]


In [7]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 128), nn.ReLU(True), nn.Linear(128, 64), nn.ReLU(True),
                                     nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))

        self.decoder = nn.Sequential(nn.Linear(3, 12), nn.ReLU(True), nn.Linear(12, 64), nn.ReLU(True),
                                     nn.Linear(64, 128), nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# **Training Setup**


In [8]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [9]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './mlp_img/image_{}.png'.format(epoch))

epoch [1/100], loss:0.1959
epoch [2/100], loss:0.1655
epoch [3/100], loss:0.1581
epoch [4/100], loss:0.1657
epoch [5/100], loss:0.1649
epoch [6/100], loss:0.1398
epoch [7/100], loss:0.1412
epoch [8/100], loss:0.1482
epoch [9/100], loss:0.1543
epoch [10/100], loss:0.1397
epoch [11/100], loss:0.1370
epoch [12/100], loss:0.1360
epoch [13/100], loss:0.1359
epoch [14/100], loss:0.1287
epoch [15/100], loss:0.1346
epoch [16/100], loss:0.1384
epoch [17/100], loss:0.1342
epoch [18/100], loss:0.1295
epoch [19/100], loss:0.1362
epoch [20/100], loss:0.1313
epoch [21/100], loss:0.1402
epoch [22/100], loss:0.1330
epoch [23/100], loss:0.1236
epoch [24/100], loss:0.1255
epoch [25/100], loss:0.1243
epoch [26/100], loss:0.1396
epoch [27/100], loss:0.1365
epoch [28/100], loss:0.1322
epoch [29/100], loss:0.1346
epoch [30/100], loss:0.1226
epoch [31/100], loss:0.1272
epoch [32/100], loss:0.1330
epoch [33/100], loss:0.1313
epoch [34/100], loss:0.1272
epoch [35/100], loss:0.1363
epoch [36/100], loss:0.1389
e